In [27]:
%load_ext autoreload
%autoreload 2

import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from registration_utilities import *
from annotation_utilities import *
from metadata import *
from data_manager import *

import matplotlib.pyplot as plt
%matplotlib inline

from joblib import Parallel, delayed
import time

No vtk


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
from registration_utilities import find_contour_points

In [29]:
stack_fixed = 'MD592'

In [30]:
train_sample_scheme = 1
global_transform_scheme = 1

# stack_moving = 'atlas_on_MD589'
stack_moving = 'atlasV2'

In [31]:
paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', 'SNC', 'SNR', '3N', '4N',
                    'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']
structures = paired_structures + singular_structures

structures_sided = sum([[n] if n in singular_structures 
                        else [convert_to_left_name(n), convert_to_right_name(n)] 
                        for n in structures], [])

In [32]:
for_local = True
local_transform_scheme = 2

In [33]:
# Read transformed volumes

if for_local:
    vols_m_aligned_to_f = {name_s: DataManager.load_transformed_volume(stack_m=stack_moving, type_m='score',
                                         stack_f=stack_fixed, type_f='score',
                                         label=name_s, downscale=32,
                                         train_sample_scheme_f=train_sample_scheme,
                                         global_transform_scheme=global_transform_scheme,
                                        local_transform_scheme=local_transform_scheme)
                        for name_s in structures_sided}

else:
    vols_m_aligned_to_f = {name_s: DataManager.load_transformed_volume(stack_m=stack_moving, type_m='score',
                                             stack_f=stack_fixed, type_f='score',
                                             label=name_s, downscale=32,
                                             train_sample_scheme_f=train_sample_scheme,
                                             global_transform_scheme=global_transform_scheme)
                            for name_s in structures_sided}



No vtk


In [34]:
# Set colors for different contour level
levels = [0.1, 0.25, 0.5, 0.75, .99]
level_colors = {level: (int(level*255),0,0) for level in levels}

In [35]:
# Set output folder

if for_local:
    viz_dir = create_if_not_exists(DataManager.get_local_alignment_viz_dir(stack_moving=stack_moving,
                                                        stack_fixed=stack_fixed,
                                                        train_sample_scheme=train_sample_scheme,
                                                        global_transform_scheme=global_transform_scheme,
                                                        local_transform_scheme=local_transform_scheme))
else:
    viz_dir = create_if_not_exists(DataManager.get_global_alignment_viz_dir(stack_moving=stack_moving,
                                                        stack_fixed=stack_fixed,
                                                        train_sample_scheme=train_sample_scheme,
                                                        global_transform_scheme=global_transform_scheme))    

In [36]:
# Generate overlay visualization

# estimate mapping between z and section
downsample_factor = 32
xy_pixel_distance_downsampled = xy_pixel_distance_lossless * downsample_factor
voxel_z_size = section_thickness / xy_pixel_distance_downsampled

# For getting correct contour location
xmin_vol_f, xmax_vol_f, ymin_vol_f, ymax_vol_f, zmin_vol_f, zmax_vol_f = \
DataManager.load_volume_bbox(stack=stack_fixed, type='score', label='7N', downscale=32)
print xmin_vol_f, xmax_vol_f, ymin_vol_f, ymax_vol_f, zmin_vol_f, zmax_vol_f

# Generate atlas overlay image for every section
first_sec, last_sec = metadata_cache['section_limits'][stack_fixed]
bar = show_progress_bar(first_sec, last_sec)

for sec in range(first_sec, last_sec+1):
    
    if metadata_cache['sections_to_filenames'][stack_fixed][sec] in ['Placeholder', 'Rescan', 'Nonexisting']:
        continue
    
    bar.value = sec

    img_fn = DataManager.get_image_filepath(stack=stack_fixed, section=sec, resol='thumbnail', version='cropped_tif')
    img = imread(img_fn)
    
    viz = img.copy()
    
    z = voxel_z_size * (sec - 1) - zmin_vol_f
    
    # Find moving volume annotation contours
    for name_s, vol in vols_m_aligned_to_f.iteritems():
        for level in levels:
            cnts = find_contours(vol[..., z], level=level) # rows, cols
            for cnt in cnts:
                # r,c to x,y
                cnt_on_cropped = cnt[:,::-1] + (xmin_vol_f, ymin_vol_f)
                cv2.polylines(viz, [cnt_on_cropped.astype(np.int)], True, level_colors[level], 2)
                

    viz_fn = os.path.join(viz_dir, '%(stack_moving)s_to_%(stack_fixed)s_%(sec)04d.jpg' % \
          {'stack_moving': stack_moving, 'stack_fixed': stack_fixed, 'sec': sec})
    imsave(viz_fn, viz)

0 544 0 386 122 505


/oasis/projects/nsf/csd395/yuncong/brain_virtualenv/lib/python2.7/site-packages/ipykernel/__main__.py:33: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
